In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

from time import time

In [6]:
columns = ['','text_no_tag', 'polarity', 'length']

dftrain = pd.read_csv('csv/finaltest.csv',
                     header = 0, 
                     usecols = [1,2], 
                     names=columns,
                     encoding ='ISO-8859-1')

In [7]:
dftrain.head()

,text_no_tag,polarity
0,@USER i love my kindle2 . not that the is co...,4
1,reading my kindle2 . love it . lee childs is...,4
2,"ok , first assesment of the kindle 2 . it fuc...",4
3,@USER you will love your kindle2 . i have had ...,4
4,@USER fair enough . but i have the kindle2 and...,4


In [13]:
tweets = []
for t in dftrain['text_no_tag']:
    #print(df_len_tweets.text[i].replace(s, ''))
    tweets.append(t)

In [14]:
str(tweets)

'[\'@USER i love  my kindle2 . not that the  is cool , but the  is fantastic in its own right .\', \'reading my kindle2 .  love it .  lee childs is good read .\', \'ok , first assesment of the kindle 2 .  it fucking rocks ! \', \'@USER you will love your kindle2 . i have had mine for a few months and never looked back . the new big one is huge ! no need for remorse ! \', "@USER fair enough . but i have the kindle2 and i think it \' s perfect ", \'@USER no . it is too big . i am quite happy with the kindle2 .\', \'fuck this economy . i hate aig and their non loan given asses .\', \'jquery is my new best friend .\', \'loves twitter\', \'how can you not love obama ? he makes jokes about himself .\', "@USER i firmly believe that obama / pelosi have zero desire to be civil . it \' s a charade and a slogan , but they want to destroy conservatism", \'house correspondents dinner was last night whoopi , barbara & sherri went , obama got a standing ovation\', \'watchin espn .  jus seen this new 

In [17]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

words = tokenizer.tokenize(str(tweets))

In [18]:
print(words)

['USER', 'i', 'love', 'my', 'kindle2', 'not', 'that', 'the', 'is', 'cool', 'but', 'the', 'is', 'fantastic', 'in', 'its', 'own', 'right', 'reading', 'my', 'kindle2', 'love', 'it', 'lee', 'childs', 'is', 'good', 'read', 'ok', 'first', 'assesment', 'of', 'the', 'kindle', '2', 'it', 'fucking', 'rocks', 'USER', 'you', 'will', 'love', 'your', 'kindle2', 'i', 'have', 'had', 'mine', 'for', 'a', 'few', 'months', 'and', 'never', 'looked', 'back', 'the', 'new', 'big', 'one', 'is', 'huge', 'no', 'need', 'for', 'remorse', 'USER', 'fair', 'enough', 'but', 'i', 'have', 'the', 'kindle2', 'and', 'i', 'think', 'it', 's', 'perfect', 'USER', 'no', 'it', 'is', 'too', 'big', 'i', 'am', 'quite', 'happy', 'with', 'the', 'kindle2', 'fuck', 'this', 'economy', 'i', 'hate', 'aig', 'and', 'their', 'non', 'loan', 'given', 'asses', 'jquery', 'is', 'my', 'new', 'best', 'friend', 'loves', 'twitter', 'how', 'can', 'you', 'not', 'love', 'obama', 'he', 'makes', 'jokes', 'about', 'himself', 'USER', 'i', 'firmly', 'believe

In [19]:
from collections import Counter, deque
from math import log

class SOPMI:
    def __init__(self, pos_seeds, neg_seeds, words, near = 10):
        self.w_count = Counter()
        self.p_count = dict()
        self.pos_seeds = set(pos_seeds)
        self.neg_seeds = set(neg_seeds)
        window = deque()
        window_size = near*2+1
        for word in words:
            window.append(word)
            if len(window)>window_size:
                window.popleft()
            elif len(window)<window_size:
                continue
            current_word = window[near]
            if current_word in self.pos_seeds or current_word in self.neg_seeds:
                self.w_count[current_word] += 1
                if current_word not in self.p_count:
                    self.p_count[current_word] = Counter()
                for window_word in window:
                    self.p_count[current_word][window_word] += 1

    def hits(self,word):
        return self.w_count[word]+1
    
    def hits_near(self,w1,w2):
        if w1 in self.p_count:
            return self.p_count[w1][w2]+1
        else:
            return 1
     
    def so(self, word):
        score = 0
        for pos_seed in self.pos_seeds:
            score += log(self.hits_near(pos_seed,word)/self.hits(pos_seed))
        for neg_seed in self.neg_seeds:
            score -= log(self.hits_near(neg_seed,word)/self.hits(neg_seed))
        return score   

In [68]:
with open('negative-words.txt', 'r', encoding='ISO-8859-1', newline='') as f:
    neg_seeds = f.read().split()
print(neg_seeds)

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive', 'abrupt', 'abruptly', 'abscond', 'absence', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'absurdly', 'absurdness', 'abuse', 'abused', 'abuses', 'abusive', 'abysmal', 'abysmally', 'abyss', 'accidental', 'accost', 'accursed', 'accusation', 'accusations', 'accuse', 'accuses', 'accusing', 'accusingly', 'acerbate', 'acerbic', 'acerbically', 'ache', 'ached', 'aches', 'achey', 'aching', 'acrid', 'acridly', 'acridness', 'acrimonious', 'acrimoniously', 'acrimony', 'adamant', 'adamantly', 'addict', 'addicted', 'addicting', 'addicts', 'admonish', 'admonisher', 'admonishingly', 'admonishment', 'admonition', 'adulterate', 'adulterated', 'adulteration', 'adulterier', 'adversarial', 'adversary', 'adverse', 'adversity', 'afflict', 'affliction', 'afflictive', 'affront', 'afraid', 'aggravate', 'aggravating', 'aggravation', 'aggression', 'aggress

In [69]:
with open('positive-words.txt', 'r', encoding='ISO-8859-1', newline='') as f:
    pos_seeds = f.read().split()
print(pos_seeds)
#pos_seeds = ['good','nice','excellent','positive','fortunate','correct','superior']
#neg_seeds = ['bad','nasty','poor','negative','unfortunate','wrong','inferior']

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [70]:
model = SOPMI(pos_seeds, neg_seeds, words)

In [71]:
model.so('social')

16.704696687392016

In [77]:
model.so('torture')

13.23896078459229

In [78]:
from nltk.corpus import stopwords

sw = stopwords.words('english')

In [79]:
vocab = set(words)
so_values = [(word, model.so(word)) for word in vocab if word not in pos_seeds and word not in neg_seeds and word not in sw]
so_values.sort(key = lambda x: x[1])

In [80]:
so_values[:50]

[('warner', -2.14071134093642),
 ('time', -1.0545215722668972),
 ('thing', 7.693783340112732),
 ('gm', 7.847934019940002),
 ('insects', 8.556829557468085),
 ('could', 9.36775977368442),
 ('cable', 9.537658810479815),
 ('dentist', 10.040287667041634),
 ('life', 10.466372062352496),
 ('know', 11.159519242912454),
 ('iran', 11.447201315364216),
 ('api', 11.447201315364243),
 ('tcot', 11.44720131536425),
 ('internet', 11.61710035215965),
 ('come', 11.85266642347242),
 ('cheney', 12.140348495924167),
 ('phone', 12.140348495924178),
 ('n', 12.545813604032334),
 ('fan', 12.545813604032341),
 ('cant', 12.545813604032348),
 ('back', 12.715712640827752),
 ('ah', 12.83349567648412),
 ('tattoos', 12.833495676484127),
 ('gop', 12.833495676484127),
 ('make', 12.83349567648413),
 ('latex', 13.084810104765056),
 ('north', 13.1211777489359),
 ('korea', 13.1211777489359),
 ('safeway', 13.12117774893591),
 ('hours', 13.121177748935914),
 ('hey', 13.238960784592287),
 ('revision', 13.238960784592287),
 ('

In [81]:
so_values[-50:]

[('lakers', 19.189603337180035),
 ('trek', 19.307386372836408),
 ('today', 19.30738637283641),
 ('gladwell', 19.307386372836415),
 ('lots', 19.477285409631794),
 ('think', 19.477285409631794),
 ('everyone', 19.477285409631794),
 ('warren', 19.477285409631808),
 ('oh', 19.477285409631808),
 ('hamilton', 19.477285409631815),
 ('big', 19.477285409631815),
 ('star', 19.595068445288195),
 ('conference', 19.882750517739964),
 ('forever', 19.882750517739964),
 ('getting', 19.882750517739964),
 ('american', 19.882750517739964),
 ('danny', 20.00053355339633),
 ('kobe', 20.00053355339635),
 ('india', 20.00053355339635),
 ('seeing', 20.170432590191737),
 ('jquery', 20.170432590191744),
 ('site', 20.288215625848128),
 ('got', 20.693680733956267),
 ('little', 20.863579770751684),
 ('io', 20.98136280640807),
 ('watching', 20.98136280640808),
 ('thanks', 20.98136280640809),
 ('weekend', 21.26904487885986),
 ('place', 21.269044878859862),
 ('kindle', 21.269044878859873),
 ('lebron', 21.31783504302929)

In [82]:
len(so_values)

1207

In [83]:
#[x[0] for x in my_tuples]
scores = []
for t in dftrain['text_no_tag']:
    score = 0
    for word in t.split():
        for i in range(0,len(so_values)):
            if word == so_values[i][0]:
                #print(so_values[i][1])
                score = score + so_values[i][1]
    scores.append(score)

In [84]:
print(scores)

[28.541443271429912, 107.48465867748763, 108.66372932672786, 215.5606013393133, 48.0187286810617, 65.1288904765619, 93.70217342686082, 69.3931340755793, 13.291076785731303, 57.56881001154006, 177.3434398994774, 216.58219547175673, 170.01103095107146, 82.0726505547026, 180.7842736471913, 187.9072208077023, 21.31783504302929, 102.06872975774962, 38.02253173042132, 167.95530593600904, 54.72722841781335, 122.64462745604953, 128.42869680881142, 104.09938113525999, 39.88678139057232, 119.3000004258758, 175.07705096818776, 111.31853501331126, 250.61607378877926, 151.75346670273376, 189.6785895872826, 189.12241400369453, 53.81888935572454, 0, 47.62918341238807, 136.46340673589478, 159.54013092341714, 94.15625721667205, 59.25870628454628, 42.48947107601668, 125.53362560314707, 131.50601595631733, 115.08900134137706, 162.88808379056056, 40.122347461885056, 70.12704613557098, 231.3660556476455, 56.418081721732946, 111.6073455991428, 101.07643501565269, 108.67964916616239, 253.5537600644357, 40.74

In [85]:
len(scores)

359

In [86]:
dftrain['SO_PMI_score'] = scores

In [87]:
dftrain.head()

,text_no_tag,polarity,SO_PMI_score
0,@USER i love my kindle2 . not that the is co...,4,28.541443
1,reading my kindle2 . love it . lee childs is...,4,107.484659
2,"ok , first assesment of the kindle 2 . it fuc...",4,108.663729
3,@USER you will love your kindle2 . i have had ...,4,215.560601
4,@USER fair enough . but i have the kindle2 and...,4,48.018729


In [88]:
dftrain.tail()

,text_no_tag,polarity,SO_PMI_score
354,"after using latex a lot , any other typeset ma...",4,92.161728
355,"on that note , i hate word . i hate pages . i ...",0,113.474864
356,ah . back in a *real* text editing environme...,4,81.123489
357,"trouble in iran , i see . hmm . iran . iran so...",0,162.264101
358,reading the tweets coming out of iran . the w...,0,106.065665
